In [4]:
import torch
from transformers import AutoTokenizer

In [6]:
train_path = 'data/20_000_examples_train.pt'
eval_path = 'data/20_000_examples_eval.pt'
tokenizer = AutoTokenizer.from_pretrained('AI-Sweden-Models/gpt-sw3-6.7b')

train_data = torch.load(train_path)
eval_data = torch.load(eval_path)



39.708333333333336


In [1]:
# Load huggingface datset
from datasets import load_dataset
dataset = load_dataset("teknium/OpenHermes-2.5")

/home/edgelab/Tim/miniconda3/envs/pytorch-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 1001551 examples [00:21, 46951.74 examples/s]


In [3]:
# Save dataset as jsonl
dataset['train'].to_json('OpenHermes-2.5.jsonl', orient='records', lines=True)


Creating json from Arrow format: 100%|██████████| 1002/1002 [00:15<00:00, 62.83ba/s]


1974353791

In [ ]:
import torch
from transformers import pipeline, StoppingCriteriaList, StoppingCriteria

device = "cuda" if torch.cuda.is_available() else "cpu"


# (Optional) - define a stopping criteria
# We ideally want the model to stop generate once the response from the Bot is generated
class StopOnTokenCriteria(StoppingCriteria):
    def __init__(self, stop_token_id):
        self.stop_token_id = stop_token_id

    def __call__(self, input_ids, scores, **kwargs):
        return input_ids[0, -1] == self.stop_token_id


pipe = pipeline(
    task="text-generation",
    model="AI-Sweden-Models/gpt-sw3-6.7b-v2-translator",
    device=device
)

stop_on_token_criteria = StopOnTokenCriteria(stop_token_id=pipe.tokenizer.bos_token_id)
text = "I like to eat ice cream in the summer."

# This will translate English to Swedish
# To translate from Swedish to English the prompt would be:
# prompt = f"<|endoftext|><s>User: Översätt till Engelska från Svenska\n{text}<s>Bot:"

prompt = f"<|endoftext|><s>User: Översätt till Svenska från Engelska\n{text}<s>Bot:"

input_tokens = pipe.tokenizer(prompt, return_tensors="pt").input_ids.to(device)
max_model_length = 2048
dynamic_max_length = max_model_length - input_tokens.shape[1]

response = pipe(
    prompt,
    max_length=dynamic_max_length,
    truncation=True,
    stopping_criteria=StoppingCriteriaList([stop_on_token_criteria])
)

print(response[0]["generated_text"].split("<s>Bot: ")[-1])


In [1]:
import torch
from transformers import pipeline, StoppingCriteriaList, StoppingCriteria, AutoModelForCausalLM


# Save model
model = AutoModelForCausalLM.from_pretrained("AI-Sweden-Models/gpt-sw3-6.7b-v2-translator")
model.save_pretrained("gpt-sw3-6.7b-v2-translator")

/home/edgelab/Tim/miniconda3/envs/pytorch-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:09<00:00,  3.07s/it]
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 2048}


In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("AI-Sweden-Models/gpt-sw3-6.7b-v2-translator")
tokenizer.save_pretrained("gpt-sw3-6.7b-v2-translator")

('gpt-sw3-6.7b-v2-translator/tokenizer_config.json',
 'gpt-sw3-6.7b-v2-translator/special_tokens_map.json',
 'gpt-sw3-6.7b-v2-translator/spiece.model',
 'gpt-sw3-6.7b-v2-translator/added_tokens.json')